In [1]:
## Split fly in half using a bounded box and count pixel density in each side

import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import sys
from scipy.spatial import distance

#Only take images that have been preprocessed using the preprocessing.py program
#In this case, the images must be contained in the "photos" directory
#The files will be saved in an empty "photos_1" directory

images = [(file, cv2.imread(file,0)) for file in glob.glob("photos/*.jpg")]
for file, img in images:
    print(file[7:])
    im2, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    blank_image = np. zeros(shape= np.shape(img))
    for c in contours:
        if (len(c) > 50):
            #Define bounded rotated box
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
              
            #Determine width and height based on distance from one corner    
            dist_box = [distance.euclidean(np.asarray(box[0]),bx) for bx in box]
            sortedbox = np.argsort(dist_box)[::-1]
            
            #Create a mask to remove other chambers, look for y-coords of targeted chamber
            box_y = np.argsort(box[:,1])
            y_min = box[box_y,1][0]
            y_max = box[box_y,1][-1]
            
            #Box coordinates based on distance from one corner
            box_pts = box[sortedbox]

            img_new = img.copy()
            img_new_2 = img.copy()

            img_new[:y_min,:]=255
            img_new[y_max:,:]=255


            img_new_2[:y_min,:]=255
            img_new_2[y_max:,:]=255

            #Determine mid point of rotated rectangles, along long and short side and draw line of seperation
            midpointLong = (np.add(box_pts[3],box_pts[1])/2).astype(int)
            midpointShort = (np.add(box_pts[2],box_pts[0])/2).astype(int)
            lineSplit = cv2.line(img_new_2,(midpointLong[0],midpointLong[1]),(midpointShort[0],midpointShort[1]),(210,0,0),1)
            
            #Retrieve points along the line
            indices = np.where(lineSplit == [210])
            xCor = indices[1]
            yCor = indices[0]
            
            #Determine how to remove either half after split depending on angle of the line of seperation
            for id1, index in enumerate(yCor):
                if midpointLong[0] < midpointShort[0]:
                    img_new[:yCor[id1],:xCor[id1]]= 255
                    img_new_2[yCor[id1]:,xCor[id1]:]= 255
                else:
                    img_new[:yCor[id1],xCor[id1]:]= 255
                    img_new_2[yCor[id1]:,:xCor[id1]]= 255
            
            #Invert image to count non-zeros
            imagem_1 = cv2.bitwise_not(img_new)
            imagem_2 = cv2.bitwise_not(img_new_2) 
            nzCount_1 = cv2.countNonZero(imagem_1)    
            nzCount_2 = cv2.countNonZero(imagem_2) 
            
            #Choose side that has fewer pixels
            if nzCount_1 < nzCount_2:
                img_chosen = imagem_1.copy()
            else:
                img_chosen = imagem_2.copy()
                
            #Concatentate each chamber into a single image 
            blank_image = blank_image + img_chosen
    cv2.imwrite("photos_1/{}".format(file[7:]), blank_image)


In [7]:
## Determine centroid of the fly and estimate the direction by measuring the maximum distance from the centroid
## Both possible directions are estimated

#Only take images that have been preprocessed using the preprocessing.py program
#In this case, the images must be contained in the "photos" directory
#The files will be saved in an empty "photos_1" directory

images = [(file, cv2.imread(file,0)) for file in glob.glob("photos/*.jpg")]
for file, img in images:
    print(file[7:])
    
    im2, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(masked_img, contours, -1, (255,255,0), 3)

    for c in contours:
        if (len(c) > 60):
            M = cv2.moments(c)
            # calculate x,y coordinate of center
            if M["m00"] != 0:
                cX = M["m10"] / M["m00"]
                cY = M["m01"] / M["m00"]
            else:
                cX, cY = 0, 0
            cv2.circle(img, (int(cX),int(cY)),2, (255, 55, 55), -1)
            center = [cX, cY]
        
            #Get Distance from centroid to each point along the contour and convert to arrays
            dist =[distance.euclidean(center,pt) for pt in c]
            pts = [pt for pt in c]
            dist_arr = np.asarray(dist)
            pts_arr = np.asarray(pts)
            
            
            #Sort distance by length in descending order and create sorted arrays for distances and pts
            sortedIdx = np.argsort(dist)[::-1]
            sortedDist = dist_arr[sortedIdx]
            sortedpts = pts_arr[sortedIdx]

        
            #Go through distances and save maximum distance from centroid in both directions (positive x and negative x
            #relative to the centroid position)
            
            
            point_retained = []
            NegativeXdirection = True
            for indexD, d in enumerate(sortedDist):
                #Save first point and determine if next point should be in the  +X or -X direction relative to center
                if indexD == 0:
                    point_retained.append([sortedpts[indexD][0][0],sortedpts[indexD][0][1]])
                    if (cX-sortedpts[indexD][0][0])<0:
                        NegativeXdirection = False
                        
                else:
                    #Save first point that is in the opposite direction and then break from loop
                    if NegativeXdirection == True:
                        if(cX-sortedpts[indexD][0][0]) < 0:
                            point_retained.append([sortedpts[indexD][0][0],sortedpts[indexD][0][1]])
                            break
                    else:
                        if(cX-sortedpts[indexD][0][0]) > 0:
                            point_retained.append([sortedpts[indexD][0][0],sortedpts[indexD][0][1]])
                            break

            
            ## First point should heading direction
            pX_1 = point_retained[0][0]
            pY_1 = point_retained[0][1]
            
            ## Second point should be tail
            pX_2 = point_retained[1][0]
            pY_2 = point_retained[1][1]

            cv2.line(img,(int(cX),int(cY)),(pX_1,pY_1),(255,0,0),1)
            
            #Uncomment if you want both directions to be shown
            #cv2.line(img,(int(cX),int(cY)),(pX_2,pY_2),(255,0,0),1)

            cv2.imwrite("photos_1/{}".format(file[7:]), img)

result1.jpg
result10.jpg
result100.jpg
result101.jpg
result102.jpg
result103.jpg
result104.jpg
result105.jpg
result106.jpg
result107.jpg
result108.jpg
result109.jpg
result11.jpg
result110.jpg
result111.jpg
result112.jpg
result113.jpg
result114.jpg
result115.jpg
result116.jpg
result117.jpg
result118.jpg
result119.jpg
result12.jpg
result120.jpg
result121.jpg
result122.jpg
result123.jpg
result124.jpg
result125.jpg
result126.jpg
result127.jpg
result128.jpg
result129.jpg
result13.jpg
result130.jpg
result131.jpg
result132.jpg
result133.jpg
result134.jpg
result135.jpg
result136.jpg
result137.jpg
result138.jpg
result139.jpg
result14.jpg
result140.jpg
result141.jpg
result142.jpg
result143.jpg
result144.jpg
result145.jpg
result146.jpg
result147.jpg
result148.jpg
result149.jpg
result15.jpg
result150.jpg
result151.jpg
result152.jpg
result153.jpg
result154.jpg
result155.jpg
result156.jpg
result16.jpg
result17.jpg
result18.jpg
result19.jpg
result2.jpg
result20.jpg
result21.jpg
result22.jpg
result23.